<a href="https://colab.research.google.com/github/SafeiaSherif/SafeiaSherif/blob/main/Multi_objective_weighted_sum_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Facility Location-Allocation Model For Disaster Management**

**This notebook is to solve a multi-obective constrained mathematical model with the weighted sum method using specific weights for each objective.**


The model is in the field of disaster management. Let's assume a natural disaster that caused a number of affected areas. These areas will contain 2 types of demand. The first one is the victims' demand(people who need medical support) and the second one is the non-victims' demand(people who need living
material support).


*   The victims' demand will be satisfied be hospitals and temporary emergency centers
*   The non-victims' demand will be satisfied by shelters.


*   The previous facilities(hospitals, centers and shelters) will need resources to serve their demand. So, it is important to add warehouses for the facilities demand.


Then, we have number of facilities need to be located and different demands need to be allocated. This problem will be solved to optimize 5 objectives subject to some constraints.

**The objectives are:**

*   Maximize the total covered population by hospitals, centers and shelters.
*   Minimize the total distances between the affected areas and the facilities.
*   Maximize the total satisfied resources' demand by warehouses.
*   Minimize the total distances between warehouses and the other facilities.
*   Minimize the total located facilities to minimize the cost.

**The main contraints:**

*   Budget constraints.
*   Capacities constraints.
*   Service distance constraints.

**Mathematical formulation and illustrative figures:**

https://docs.google.com/document/d/1f3Y7rZNJTPW-5FSfTnqebAyqIGR5AkoEoYmtPYpHUqc/edit?usp=sharing

In [1]:
import pandas as pd
import numpy as np
import math

As in the mathematical model, we need to assume some parameters and constraints.
The next part contain those assumbtions.

We assumed 3 disaster zones, 2 valid hospitals, 2 potential places for centers, 2 potential places for shelters and 2 potential places for warehouses.


*   Each facility and each zone has (x, y) location.
*   I used the role of the distance between 2 points to calculate the distances between each facility and each disaster zone.



In [2]:
scl = 19  # scaling parameter used to scale the distances to suitable values in Km

In [3]:
i1 = [-0.8, 0.8]     # (x, y) location for the first disaster zone
i2 = [1.6, 1.6]      # (x, y) location for the second disaster zone
i3 = [1.2, -0.6]     # (x, y) location for the third disaster zone

j1 = [1.3, 0.5]      # (x, y) location for the first hospital
j2 = [0.4, 1.3]      # (x, y) location for the second hospital

k1 = [2.05, 0.3]     # (x, y) location for the first center
k2 = [-0.13, -0.23]  # (x, y) location for the second center

f1 = [0.42, 0.45]    # (x, y) location for the first shelter
f2 = [1.85, 0.65]    # (x, y) location for the second shelter

z1 = [0.4, 0.2]      # (x, y) location for the first warehouse
z2 = [0.83, 1.3]     # (x, y) location for the second warehouse

Here I calculate the 2d matrix of the distances between the hospitals and the disaster zones

In [4]:
D_H = [0]*6

D_H[0] = math.sqrt((i1[0]-j1[0])**2+(i1[1]-j1[1])**2)
D_H[1] = math.sqrt((i2[0]-j1[0])**2+(i2[1]-j1[1])**2)
D_H[2] = math.sqrt((i3[0]-j1[0])**2+(i3[1]-j1[1])**2)
D_H[3] = math.sqrt((i1[0]-j2[0])**2+(i1[1]-j2[1])**2)
D_H[4] = math.sqrt((i2[0]-j2[0])**2+(i2[1]-j2[1])**2)
D_H[5] = math.sqrt((i3[0]-j2[0])**2+(i3[1]-j2[1])**2)

D_H = np.array(D_H).reshape(2, 3)

D_H = D_H*scl
D_H = D_H.astype(int)
D_H

array([[40, 21, 20],
       [24, 23, 39]])

Here I calculate the 2d matrix of the distances between the centers and the disaster zones


In [5]:
D_C = [0]*6

D_C[0] = math.sqrt((i1[0]-k1[0])**2+(i1[1]-k1[1])**2)
D_C[1] = math.sqrt((i2[0]-k1[0])**2+(i2[1]-k1[1])**2)
D_C[2] = math.sqrt((i3[0]-k1[0])**2+(i3[1]-k1[1])**2)
D_C[3] = math.sqrt((i1[0]-k2[0])**2+(i1[1]-k2[1])**2)
D_C[4] = math.sqrt((i2[0]-k2[0])**2+(i2[1]-k2[1])**2)
D_C[5] = math.sqrt((i3[0]-k2[0])**2+(i3[1]-k2[1])**2)

D_C = np.array(D_C).reshape(2, 3)

D_C = D_C*scl
D_C = D_C.astype(int)
D_C

array([[54, 26, 23],
       [23, 47, 26]])

Here I calculate the 2d matrix of the distances between the shelters and the disaster zones

In [6]:
D_SH = [0]*6

D_SH[0] = math.sqrt((i1[0]-f1[0])**2+(i1[1]-f1[1])**2)
D_SH[1] = math.sqrt((i2[0]-f1[0])**2+(i2[1]-f1[1])**2)
D_SH[2] = math.sqrt((i3[0]-f1[0])**2+(i3[1]-f1[1])**2)
D_SH[3] = math.sqrt((i1[0]-f2[0])**2+(i1[1]-f2[1])**2)
D_SH[4] = math.sqrt((i2[0]-f2[0])**2+(i2[1]-f2[1])**2)
D_SH[5] = math.sqrt((i3[0]-f2[0])**2+(i3[1]-f2[1])**2)

D_SH = np.array(D_SH).reshape(2, 3)

D_SH = D_SH*scl
D_SH = D_SH.astype(int)
D_SH

array([[24, 31, 24],
       [50, 18, 26]])

Here I calculate the 2d matrix of the distances between the hospitals and the warehouses

In [7]:
D_H_W = [0]*4

D_H_W[0] = math.sqrt((z1[0]-j1[0])**2+(z1[1]-j1[1])**2)
D_H_W[1] = math.sqrt((z2[0]-j1[0])**2+(z2[1]-j1[1])**2)
D_H_W[2] = math.sqrt((z1[0]-j2[0])**2+(z1[1]-j2[1])**2)
D_H_W[3] = math.sqrt((z2[0]-j2[0])**2+(z2[1]-j2[1])**2)

D_H_W = np.array(D_H_W).reshape(2, 2)

D_H_W = D_H_W*scl
D_H_W = D_H_W.astype(int)
D_H_W

array([[18, 17],
       [20,  8]])

Here I calculate the 2d matrix of the distances between the centers and the warehouses


In [8]:
D_C_W = [0]*4

D_C_W[0] = math.sqrt((z1[0]-k1[0])**2+(z1[1]-k1[1])**2)
D_C_W[1] = math.sqrt((z2[0]-k1[0])**2+(z2[1]-k1[1])**2)
D_C_W[2] = math.sqrt((z1[0]-k2[0])**2+(z1[1]-k2[1])**2)
D_C_W[3] = math.sqrt((z2[0]-k2[0])**2+(z2[1]-k2[1])**2)

D_C_W = np.array(D_C_W).reshape(2, 2)

D_C_W = D_C_W*scl
D_C_W = D_C_W.astype(int)
D_C_W

array([[31, 29],
       [12, 34]])

Here I calculate the 2d matrix of the distances between the shelters and the warehouses

In [9]:
D_SH_W = [0]*4

D_SH_W[0] = math.sqrt((z1[0]-f1[0])**2+(z1[1]-f1[1])**2)
D_SH_W[1] = math.sqrt((z2[0]-f1[0])**2+(z2[1]-f1[1])**2)
D_SH_W[2] = math.sqrt((z1[0]-f2[0])**2+(z1[1]-f2[1])**2)
D_SH_W[3] = math.sqrt((z2[0]-f2[0])**2+(z2[1]-f2[1])**2)

D_SH_W = np.array(D_SH_W).reshape(2, 2)

D_SH_W = D_SH_W*scl
D_SH_W = D_SH_W.astype(int)
D_SH_W

array([[ 4, 17],
       [28, 22]])

**Here I specify the limits or the right hand side of the constraints**

*   The capacities of the facilities.
*   The population of the affected areas.
*   The severity of the disaster (percentage of victims) for each zone.
*   Units of resources needed by one victim and so for the non-victim.
*   Maximum service distances for each kind of facilities.
*   Maximum allowed number of located facilities for each kind of facilities.







In [10]:
Cap_H = np.array([750, 230])
Cap_C = np.array([100, 60])
Cap_SH = np.array([700, 600])
Cap_V_W = np.array([300, 1000])
Cap_NV_W = np.array([1000, 600])

a = np.array([1000,1000,1000])
# p = np.array([0.8, 0.7, 0.6])
p = np.array([0.1, 0.07, 0.19])
victim = a * p
victim = np.array(victim).astype(int)
non_victim = a * (1-p)
non_victim = np.array(non_victim).astype(int)

U_R_V = 2
U_R_NV = 2

max_D_H = 40
max_D_C = 40
max_D_SH = 40
max_D_W = 40

N = 1 # Maximum allowed number of located centers
M = 2 # Maximum allowed number of located shelters
Q = 2 # Maximum allowed number of located warehouses

**Here I install the solver to use it. I used Gurobi solver as it's suitable for the integer linear problems.**

In [11]:
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 43.3 MB/s eta 0:00:00


In [12]:
from itertools import product

In [13]:
# create a new Integer Linear Programming (ILP) solver
m = gp.Model("ILP")

Restricted license - for non-production use only - expires 2024-10-28


In [14]:
num_hospitals = 2
num_centers = 2
num_shelters = 2
num_nodes = 3
num_warehouses = 2

cartesian_prod_hos = list(product(range(num_hospitals), range(num_nodes)))
cartesian_prod_cntr = list(product(range(num_centers), range(num_nodes)))
cartesian_prod_shltr = list(product(range(num_shelters), range(num_nodes)))
cartesian_prod_warehouse_H = list(product(range(num_warehouses), range(num_hospitals)))
cartesian_prod_warehouse_C = list(product(range(num_warehouses), range(num_centers)))
cartesian_prod_warehouse_SH = list(product(range(num_warehouses), range(num_shelters)))


# **Initialization of the decisoin variables**

The decision variables are illustrated in this link

https://docs.google.com/document/d/1f3Y7rZNJTPW-5FSfTnqebAyqIGR5AkoEoYmtPYpHUqc/edit?usp=sharing

In [15]:

exist_H = m.addVars(num_hospitals,vtype=GRB.BINARY, name="exist_H")
exist_C = m.addVars(num_centers,vtype=GRB.BINARY, name="exist_C")
exist_SH = m.addVars(num_shelters,vtype=GRB.BINARY, name="exist_SH")

S_D_H = m.addVars(cartesian_prod_hos, lb=0, vtype=GRB.INTEGER, name="S_D_H")
S_D_C = m.addVars(cartesian_prod_cntr, lb=0, vtype=GRB.INTEGER, name="S_D_C")
S_D_SH = m.addVars(cartesian_prod_shltr, lb=0, vtype=GRB.INTEGER, name="S_D_SH")

assigned_H = m.addVars(cartesian_prod_hos,vtype=GRB.BINARY, name="assigned_H")
assigned_C = m.addVars(cartesian_prod_cntr,vtype=GRB.BINARY, name="assigned_C")
assigned_SH = m.addVars(cartesian_prod_shltr,vtype=GRB.BINARY, name="assigned_SH")

exist_W = m.addVars(num_warehouses,vtype=GRB.BINARY, name="exist_W")

S_D_H_W = m.addVars(cartesian_prod_warehouse_H, lb=0, vtype=GRB.INTEGER, name="S_D_H_W")
S_D_C_W = m.addVars(cartesian_prod_warehouse_C, lb=0, vtype=GRB.INTEGER, name="S_D_C_W")
S_D_SH_W = m.addVars(cartesian_prod_warehouse_SH, lb=0, vtype=GRB.INTEGER, name="S_D_SH_W")

assigned_H_W =  m.addVars(cartesian_prod_warehouse_H,vtype=GRB.BINARY, name="assigned_H_W")
assigned_C_W =  m.addVars(cartesian_prod_warehouse_C,vtype=GRB.BINARY, name="assigned_C_W")
assigned_SH_W =  m.addVars(cartesian_prod_warehouse_SH,vtype=GRB.BINARY, name="assigned_SH_W")

sum_S_D_H = m.addVars(num_hospitals, lb=0, vtype=GRB.INTEGER, name="sum_S_D_H")  # This DV is not basic in the model, it is only used to make summation for other DVs.
sum_S_D_C = m.addVars(num_centers, lb=0, vtype=GRB.INTEGER, name="sum_S_D_C")    # This DV is not basic in the model, it is only used to make summation for other DVs.
sum_S_D_SH = m.addVars(num_shelters, lb=0, vtype=GRB.INTEGER, name="sum_S_D_SH") # This DV is not basic in the model, it is only used to make summation for other DVs.

# **Distance constraints**

In [16]:
# add constraints of distances[D_H(j,i)*assigned_H(j,i) <= max_D_H],[D_C(k,i)*assigned_C(k,i) <= max_D_C],[D_SH(f,i)*assigned_SH(f,i) <= max_D_SH]
m.addConstrs((D_H[(j,i)]*assigned_H[(j,i)] <= max_D_H for j,i in cartesian_prod_hos), name="max_Dis_Hos")
m.addConstrs((D_C[(k,i)]*assigned_C[(k,i)] <= max_D_C for k,i in cartesian_prod_cntr), name="max_Dis_Cntr")
m.addConstrs((D_SH[(f,i)]*assigned_SH[(f,i)] <= max_D_SH for f,i in cartesian_prod_shltr), name="max_Dis_Shltr")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>}

In [17]:
m.addConstrs((D_H_W[(z,j)]*assigned_H_W[(z,j)] <= max_D_W for z,j in cartesian_prod_warehouse_H), name="max_dis_warehouse_Hos")
m.addConstrs((D_C_W[(z,k)]*assigned_C_W[(z,k)] <= max_D_W for z,k in cartesian_prod_warehouse_C), name="max_dis_warehouse_Cntr")
m.addConstrs((D_SH_W[(z,f)]*assigned_SH_W[(z,f)] <= max_D_W for z,f in cartesian_prod_warehouse_SH), name="max_dis_warehouse_Shltr")


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>}

# **Capacity constraints**

In [18]:
# add constraints of capacities [∑ S_D_H(j,i) <= Cap_H(j)*exist_H],[∑ S_D_C(k,i) <= Cap_C(k)*exist_C],[∑ S_D_SH(f,i) <= Cap_SH(f)*exist_SH]
m.addConstrs((gp.quicksum(S_D_H[(j,i)] for i in range(num_nodes)) <= Cap_H[j]*exist_H[j] for j in range(num_hospitals)), name="Cap_Hos")
m.addConstrs((gp.quicksum(S_D_C[(k,i)] for i in range(num_nodes)) <= Cap_C[k]*exist_C[k] for k in range(num_centers)), name="Cap_Cntr")
m.addConstrs((gp.quicksum(S_D_SH[(f,i)] for i in range(num_nodes)) <= Cap_SH[f]*exist_SH[f] for f in range(num_shelters)), name="Cap_Shltr")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [19]:
m.addConstrs(((gp.quicksum(S_D_H_W[(z,j)] for j in range(num_hospitals)) + gp.quicksum(S_D_C_W[(z,k)] for k in range(num_centers))) <= Cap_V_W[z]*exist_W[z] for z in range(num_warehouses)), name="Cap_warehouse_victim")
m.addConstrs((gp.quicksum(S_D_SH_W[(z,f)] for f in range(num_shelters)) <= Cap_NV_W[z]*exist_W[z] for z in range(num_warehouses)), name="Cap_warehouse_nonvictim")


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

# **Actual demand constraints**

In [20]:
# add constraint to meet the actual demand [∑ S_D_H(j,i) + ∑ S_D_C(k,i) <= ai*pi],[∑ S_D_SH(f,i) <= ai*(1-pi)]
m.addConstrs(((gp.quicksum(S_D_H[(j,i)] for j in range(num_hospitals)) + gp.quicksum(S_D_C[(k,i)] for k in range(num_centers))) <= victim[i] for i in range(num_nodes)), name="acual_victim_demand")
m.addConstrs((gp.quicksum(S_D_SH[(f,i)] for f in range(num_shelters)) <= non_victim[i] for i in range(num_nodes)), name="acual_non_victim_demand")


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [21]:
# The summation of a hospital satisfied demand will be a part of the right hand side of the warehouse actual demand.
m.addConstrs((gp.quicksum(S_D_H[(j,i)] for i in range(num_nodes)) == sum_S_D_H[j] for j in range(num_hospitals)), name="Sum_S_D_Hos")

# The summation of a center satisfied demand will be a part of the right hand side of the warehouse actual demand.
m.addConstrs((gp.quicksum(S_D_C[(k,i)] for i in range(num_nodes)) == sum_S_D_C[k] for k in range(num_centers)), name="Sum_S_D_Hntr")

# The summation of a shelter satisfied demand will be a part of the right hand side of the warehouse actual demand.
m.addConstrs((gp.quicksum(S_D_SH[(f,i)] for i in range(num_nodes)) == sum_S_D_SH[f] for f in range(num_shelters)), name="Sum_S_D_Shltr")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [22]:
m.addConstrs((gp.quicksum(S_D_H_W[(z,j)] for z in range(num_warehouses)) <= U_R_V * sum_S_D_H[j]*exist_H[j] for j in range(num_hospitals)), name="satisfy_actual_demand_hospital")
m.addConstrs((gp.quicksum(S_D_C_W[(z,k)] for z in range(num_warehouses)) <= U_R_V * sum_S_D_C[k]*exist_C[k] for k in range(num_centers)), name="satisfy_demand_Center")
m.addConstrs((gp.quicksum(S_D_SH_W[(z,f)] for z in range(num_warehouses)) <= U_R_NV * sum_S_D_SH[f]*exist_SH[f] for f in range(num_shelters)), name="satisfy_demand_Shelter")

{0: <gurobi.QConstr Not Yet Added>, 1: <gurobi.QConstr Not Yet Added>}

# **Maximum number of located facilities constraints**

In [24]:

m.addConstr((gp.quicksum(exist_C) <= N), name="limited_Cntr")
m.addConstr((gp.quicksum(exist_SH) <= M), name="limited_Shltr")
m.addConstr((gp.quicksum(exist_W) <= Q), name="limited_WarHouse")


<gurobi.Constr *Awaiting Model Update*>

In [25]:
# Those constraints are used only to make sure that the satisfied demand by any facility will take value of 1 in its assigned DV.
# The are not basic in the model. They just to make sure the solver get true solutions
m.addConstrs((S_D_H[(j,i)]*assigned_H[(j,i)] == S_D_H[(j,i)] for j,i in cartesian_prod_hos), name="additional_h")
m.addConstrs((S_D_C[(k,i)]*assigned_C[(k,i)] == S_D_C[(k,i)] for k,i in cartesian_prod_cntr), name="additional_c")
m.addConstrs((S_D_SH[(f,i)]*assigned_SH[(f,i)] == S_D_SH[(f,i)] for f,i in cartesian_prod_shltr), name="additional_sh")
m.addConstrs((S_D_H_W[(z,j)]*assigned_H_W[(z,j)] == S_D_H_W[(z,j)] for z,j in cartesian_prod_warehouse_H), name="additional_h_w")
m.addConstrs((S_D_C_W[(z,k)]*assigned_C_W[(z,k)] == S_D_C_W[(z,k)] for z,k in cartesian_prod_warehouse_C), name="additional_c_w")
m.addConstrs((S_D_SH_W[(z,f)]*assigned_SH_W[(z,f)] == S_D_SH_W[(z,f)] for z,f in cartesian_prod_warehouse_SH), name="additional_sh_w")

{(0, 0): <gurobi.QConstr Not Yet Added>,
 (0, 1): <gurobi.QConstr Not Yet Added>,
 (1, 0): <gurobi.QConstr Not Yet Added>,
 (1, 1): <gurobi.QConstr Not Yet Added>}

# **The normalization factors and weights**

*   Each objective is devided into sub-objectives. As each objective consists of many parts summed together in one objective.
*   Each sub-objective will be devided by its absolute maximum value.
*   The unit of weight is 10/181. Each sub-objective is multiplied by its level of importance
*   The summation of weights = 1.





In [26]:
norm11 = 1/(sum(Cap_H))
norm12 = 1/(sum(Cap_C))
norm13 = 1/(sum(Cap_SH))

norm21 = 1/(sum(sum(D_H)))
norm22 = 1/(sum(sum(D_C)))
norm23 = 1/(sum(sum(D_SH)))

norm31 = 1/(sum(Cap_V_W))
norm32 = 1/(sum(Cap_V_W))
norm33 = 1/(sum(Cap_NV_W))


norm41 = 1/(sum(sum(D_H_W)))
norm42 = 1/(sum(sum(D_C_W)))
norm43 = 1/(sum(sum(D_SH_W)))

norm51 = 1/num_hospitals
norm52 = 1/N
norm53 = 1/M
norm54 = 1/Q

w = 1/18.1

# **The weighted sum single objective**

In [27]:

# set objective
m.setObjective(2*w*norm11*(gp.quicksum(S_D_H[(j,i)]*assigned_H[(j,i)] for j,i in cartesian_prod_hos)) + 1.5*w*norm12*(gp.quicksum(S_D_C[(k,i)]*assigned_C[(k,i)] for k,i in cartesian_prod_cntr)) + 2*w*norm13*(gp.quicksum(S_D_SH[(f,i)]*assigned_SH[(f,i)] for f,i in cartesian_prod_shltr))
- norm21*w*(gp.quicksum(D_H[(j,i)] * assigned_H[(j,i)] for j,i in cartesian_prod_hos)) - norm22*w*(gp.quicksum(D_C[(k,i)]*assigned_C[(k,i)] for k,i in cartesian_prod_cntr )) - norm23*w*(gp.quicksum(D_SH[(f,i)]*assigned_SH[(f,i)] for f,i in cartesian_prod_shltr ))
+ 2*norm31*w*(gp.quicksum(S_D_H_W[(z,j)]*assigned_H_W[(z,j)] for z,j in cartesian_prod_warehouse_H)) + 2*w*norm32*(gp.quicksum(S_D_C_W[(z,k)]*assigned_C_W[(z,k)] for z,k in cartesian_prod_warehouse_C)) + 2*w*norm33*(gp.quicksum(S_D_SH_W[(z,f)]*assigned_SH_W[(z,f)] for z,f in cartesian_prod_warehouse_SH))
- norm41*w*(gp.quicksum(D_H_W[(z,j)]*assigned_H_W[(z,j)] for z,j in cartesian_prod_warehouse_H)) - norm42*w*(gp.quicksum(D_C_W[(z,k)] * assigned_C_W[(z,k)] for z,k in cartesian_prod_warehouse_C)) - norm43*w*(gp.quicksum(D_SH_W[(z,f)] * assigned_SH_W[(z,f)] for z,f in cartesian_prod_warehouse_SH))
- 0.2*w*norm51*(gp.quicksum(exist_H)) - 0.2*w*norm52*(gp.quicksum(exist_C)) - 0.1*w*norm53*(gp.quicksum(exist_SH)) - 0.1*w*norm54*(gp.quicksum(exist_W))
, GRB.MAXIMIZE)

In [28]:
# optimization of model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 55 rows, 74 columns and 118 nonzeros
Model fingerprint: 0x20df56a4
Model has 30 quadratic objective terms
Model has 36 quadratic constraints
Variable types: 0 continuous, 74 integer (38 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [3e-03, 2e-02]
  QObjective range [1e-04, 1e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolved: 67 rows, 74 columns, 161 nonzeros
Variable types: 0 continuous, 74 integer (35 binary)

Root relaxation: objective 2.629563e-01, 42 iterations, 0.00 seconds (0.00 work units)

    Nodes    |   

The values of the decision variables are in one list. So, I needed to separate each kind of DVs in a list so I can diplay all DVs values in a readable way.

In [29]:
# display optimal values of decision variables and objective function

vars = m.getVars()

hospitals = []
centers = []
shelters = []
warehouses = []

demand_satisfied_H = []
demand_satisfied_C = []
demand_satisfied_SH = []
demand_satisfied_H_W = []
demand_satisfied_C_W = []
demand_satisfied_SH_W = []

nodes_assigned_H = []
nodes_assigned_C = []
nodes_assigned_SH = []
nodes_assigned_H_W = []
nodes_assigned_C_W = []
nodes_assigned_SH_W = []

sum_demand_satisfied_H = []
sum_demand_satisfied_C = []
sum_demand_satisfied_SH = []

for j in range(0, num_hospitals):
  hospitals.append(vars[j].X)
hospitals = np.array(hospitals).astype(int)

for k in range(num_hospitals, num_hospitals+num_centers):
  centers.append(vars[k].X)
centers = np.array(centers).astype(int)

for f in range(num_hospitals+num_centers, num_hospitals+num_centers+num_shelters):
  shelters.append(vars[f].X)
shelters = np.array(shelters).astype(int)

for j in range(num_hospitals+num_centers+num_shelters, num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes)):
  demand_satisfied_H.append(vars[j].X)
demand_satisfied_H = np.array(demand_satisfied_H).reshape(num_hospitals, num_nodes).astype(int)
sum_victims_hospital = np.apply_along_axis(np.sum, axis=1, arr=demand_satisfied_H)
sum_victims_node_H = np.apply_along_axis(np.sum, axis=0, arr=demand_satisfied_H)
demand_satisfied_H = pd.DataFrame(demand_satisfied_H)
demand_satisfied_H["exist_H"] = hospitals
demand_satisfied_H["Sum_S_D_H"] = sum_victims_hospital
demand_satisfied_H["Cap_H"] = Cap_H

for k in range(num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes), num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes)+(num_centers*num_nodes)):
  demand_satisfied_C.append(vars[k].X)
demand_satisfied_C = np.array(demand_satisfied_C).reshape(num_centers, num_nodes).astype(int)
sum_victims_center = np.apply_along_axis(np.sum, axis=1, arr=demand_satisfied_C)
sum_victims_node_C = np.apply_along_axis(np.sum, axis=0, arr=demand_satisfied_C)
demand_satisfied_C = pd.DataFrame(demand_satisfied_C)
demand_satisfied_C["exist_C"] = centers
demand_satisfied_C["Sum_S_D_C"] = sum_victims_center
demand_satisfied_C["Cap_C"] = Cap_C

for f in range(num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes)+(num_centers*num_nodes), num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes)+(num_centers*num_nodes)+(num_shelters*num_nodes)):
  demand_satisfied_SH.append(vars[f].X)
demand_satisfied_SH = np.array(demand_satisfied_SH).reshape(num_shelters, num_nodes).astype(int)
sum_victims_shelter = np.apply_along_axis(np.sum, axis=1, arr=demand_satisfied_SH)
sum_victims_node_SH = np.apply_along_axis(np.sum, axis=0, arr=demand_satisfied_SH)
demand_satisfied_SH = pd.DataFrame(demand_satisfied_SH)
demand_satisfied_SH["exist_SH"] = shelters
demand_satisfied_SH["Sum_S_D_SH"] = sum_victims_shelter
demand_satisfied_SH["Cap_SH"] = Cap_SH

for j in range(num_hospitals+num_centers+num_shelters+(num_hospitals*num_nodes)+(num_centers*num_nodes)+(num_shelters*num_nodes), num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(num_centers*num_nodes)+(num_shelters*num_nodes)):
  nodes_assigned_H.append(vars[j].X)
nodes_assigned_H = np.array(nodes_assigned_H).reshape(num_hospitals, num_nodes).astype(int)
nodes_assigned_H = pd.DataFrame(nodes_assigned_H)

for k in range(num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(num_centers*num_nodes)+(num_shelters*num_nodes), num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(num_shelters*num_nodes)):
  nodes_assigned_C.append(vars[k].X)
nodes_assigned_C = np.array(nodes_assigned_C).reshape(num_centers, num_nodes).astype(int)
nodes_assigned_C = pd.DataFrame(nodes_assigned_C)

for f in range(num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(num_shelters*num_nodes), num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)):
  nodes_assigned_SH.append(vars[f].X)
nodes_assigned_SH = np.array(nodes_assigned_SH).reshape(num_shelters, num_nodes).astype(int)
nodes_assigned_SH = pd.DataFrame(nodes_assigned_SH)

for z in range(num_hospitals+num_centers+num_shelters+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)):
  warehouses.append(vars[z].X)
warehouses = np.array(warehouses).astype(int)

for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals)):
  demand_satisfied_H_W.append(vars[z].X)
demand_satisfied_H_W = np.array(demand_satisfied_H_W).reshape(num_warehouses, num_hospitals).astype(int)
sum_demand_H_W = np.apply_along_axis(np.sum, axis=1, arr=demand_satisfied_H_W)
sum_node_H_W = np.apply_along_axis(np.sum, axis=0, arr=demand_satisfied_H_W)
demand_satisfied_H_W = pd.DataFrame(demand_satisfied_H_W)
demand_satisfied_H_W["exist_W"] = warehouses
demand_satisfied_H_W["Sum_S_D_H_W"] = sum_demand_H_W
demand_satisfied_H_W["Cap_V_W"] = Cap_V_W

for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals),num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals)+(num_warehouses*num_centers)):
  demand_satisfied_C_W.append(vars[z].X)
demand_satisfied_C_W = np.array(demand_satisfied_C_W).reshape(num_warehouses, num_centers).astype(int)
sum_demand_C_W = np.apply_along_axis(np.sum, axis=1, arr=demand_satisfied_C_W)
sum_node_C_W = (np.apply_along_axis(np.sum, axis=0, arr=demand_satisfied_C_W))
demand_satisfied_C_W = pd.DataFrame(demand_satisfied_C_W)
demand_satisfied_C_W["exist_W"] = warehouses
demand_satisfied_C_W["Sum_S_D_C_W"] = sum_demand_C_W
demand_satisfied_C_W["Cap_V_W"] = Cap_V_W

for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals)+(num_warehouses*num_centers),num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals)+(num_warehouses*num_centers)+(num_warehouses*num_shelters)):
  demand_satisfied_SH_W.append(vars[z].X)
demand_satisfied_SH_W = np.array(demand_satisfied_SH_W).reshape(num_warehouses, num_shelters).astype(int)
sum_demand_SH_W = np.apply_along_axis(np.sum, axis=1, arr=demand_satisfied_SH_W)
sum_node_SH_W = (np.apply_along_axis(np.sum, axis=0, arr=demand_satisfied_SH_W))
demand_satisfied_SH_W = pd.DataFrame(demand_satisfied_SH_W)
demand_satisfied_SH_W["exist_W"] = warehouses
demand_satisfied_SH_W["Sum_S_D_SH_W"] = sum_demand_SH_W
demand_satisfied_SH_W["Cap_NV_W"] = Cap_NV_W

for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(num_warehouses*num_hospitals)+(num_warehouses*num_centers)+(num_warehouses*num_shelters), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(num_warehouses*num_centers)+(num_warehouses*num_shelters)):
  nodes_assigned_H_W.append(vars[z].X)
nodes_assigned_H_W = np.array(nodes_assigned_H_W).reshape(num_warehouses, num_hospitals).astype(int)
nodes_assigned_H_W = pd.DataFrame(nodes_assigned_H_W)

for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(num_warehouses*num_centers)+(num_warehouses*num_shelters), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(num_warehouses*num_shelters)):
  nodes_assigned_C_W.append(vars[z].X)
nodes_assigned_C_W = np.array(nodes_assigned_C_W).reshape(num_warehouses, num_centers).astype(int)
nodes_assigned_C_W = pd.DataFrame(nodes_assigned_C_W)

for z in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(num_warehouses*num_shelters), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)):
  nodes_assigned_SH_W.append(vars[z].X)
nodes_assigned_SH_W = np.array(nodes_assigned_SH_W).reshape(num_warehouses, num_shelters).astype(int)
nodes_assigned_SH_W = pd.DataFrame(nodes_assigned_SH_W)

for j in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters), num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)+num_hospitals):
  sum_demand_satisfied_H.append(vars[j].X)
sum_demand_satisfied_H = np.array(sum_demand_satisfied_H).astype(int)

for k in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)+num_hospitals, num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)+num_hospitals+num_centers):
  sum_demand_satisfied_C.append(vars[k].X)
sum_demand_satisfied_C = np.array(sum_demand_satisfied_C).astype(int)

for f in range(num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)+num_hospitals+num_centers, num_hospitals+num_centers+num_shelters+num_warehouses+(2*num_hospitals*num_nodes)+(2*num_centers*num_nodes)+(2*num_shelters*num_nodes)+(2*num_warehouses*num_hospitals)+(2*num_warehouses*num_centers)+(2*num_warehouses*num_shelters)+num_hospitals+num_centers+num_shelters):
  sum_demand_satisfied_SH.append(vars[f].X)
sum_demand_satisfied_SH = np.array(sum_demand_satisfied_SH).astype(int)

# Located facilities
print("Hospitals: ", hospitals)
print("Centers: ", centers)
print("Shelters: ", shelters)
print("Warehouses: ", warehouses)

# Demand satisfied by hospitals
print("Demand satisfied by each Hospital: ", "\n", demand_satisfied_H)
print(sum_victims_node_H)
print(victim, "\n")

# Demand satisfied by centers
print("Demand satisfied by each Center: ", "\n", demand_satisfied_C)
print(sum_victims_node_C)
print(sum_victims_node_H+sum_victims_node_C)
print(victim)
print("Shortage = ", sum(victim)-(sum(sum_victims_node_H)+sum(sum_victims_node_C)), "\n")

# Demand satisfied by shelters
print("Demand satisfied by each Shelter: ", "\n",demand_satisfied_SH)
print(sum_victims_node_SH)
print(non_victim)
print("Shortage = ", sum(non_victim)-sum(sum_victims_node_SH), "\n")

# Demand satisfied by warehouses for hospitals
print("Hospitals' demand satisfied by each warehouse:", "\n", demand_satisfied_H_W)
print(sum_node_H_W)
print(sum_demand_satisfied_H*U_R_V)
print("Shortage = ", U_R_V*sum(sum_demand_satisfied_H)-sum(sum_node_H_W))

# Demand satisfied by warehouses for centers
print("Centers' demand satisfied by each warehouse:", "\n", demand_satisfied_C_W)
print(sum_node_C_W)
print(sum_demand_satisfied_C*U_R_V)
print("Shortage = ", U_R_V*sum(sum_demand_satisfied_C)-sum(sum_node_C_W))

# Demand satisfied by warehouses for shelters
print("Shelters' demand satisfied by each warehouse:", "\n", demand_satisfied_SH_W)
print(sum_node_SH_W)
print(sum_demand_satisfied_SH*U_R_NV)
print("Shortage = ", U_R_NV*sum(sum_demand_satisfied_SH)-sum(sum_node_SH_W))

# disaster zones assigned to hospitals
print("Nodes assigned to each hospital: ", "\n",nodes_assigned_H, "\n")
# disaster zones assigned to centers
print("Nodes assigned to each center: ", "\n",nodes_assigned_C, "\n")
# disaster zones assigned to shelters
print("Nodes assigned to each shelter: ", "\n",nodes_assigned_SH, "\n")
# hospitals assigned to warehouses
print("Hospitals assigned to each warehouse", "\n", nodes_assigned_H_W, "\n")
# centers assigned to warehouses
print("Centers assigned to each warehouse", "\n", nodes_assigned_C_W, "\n")
# shelters assigned to warehouses
print("Shelters assigned to each warehouse", "\n", nodes_assigned_SH_W, "\n")

# objective value
print("The single Objective value = ", m.objVal)


Hospitals:  [0 1]
Centers:  [1 0]
Shelters:  [1 1]
Warehouses:  [1 1]
Demand satisfied by each Hospital:  
      0  1    2  exist_H  Sum_S_D_H  Cap_H
0    0  0    0        0          0    750
1  100  0  130        1        230    230
[100   0 130]
[100  70 190] 

Demand satisfied by each Center:  
    0   1   2  exist_C  Sum_S_D_C  Cap_C
0  0  70  30        1        100    100
1  0   0   0        0          0     60
[ 0 70 30]
[100  70 160]
[100  70 190]
Shortage =  30 

Demand satisfied by each Shelter:  
    0    1    2  exist_SH  Sum_S_D_SH  Cap_SH
0  0    0  700         1         700     700
1  0  600    0         1         600     600
[  0 600 700]
[900 929 810]
Shortage =  1339 

Hospitals' demand satisfied by each warehouse: 
    0    1  exist_W  Sum_S_D_H_W  Cap_V_W
0  0    0        1            0      300
1  0  460        1          460     1000
[  0 460]
[  0 460]
Shortage =  0
Centers' demand satisfied by each warehouse: 
      0  1  exist_W  Sum_S_D_C_W  Cap_V_W
0    0  0  